# Imports and Global Variables

In [1]:
import os
os.chdir('/Users/gil2rok/school/crispr-phylogeny2/code/Heracles')

import pickle
import torch
import geoopt
import icecream as ic
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow

from mlflow import log_metric
from scipy.linalg import expm
from scipy.cluster.hierarchy import average, to_tree
import icecream as ic

from metrics import dist_correlation, rf, tc2
from logalike import Logalike
from util.util import char_matrix_to_dist_matrix, embed_tree, estimate_tree, manifold_dist, compute_error

In [2]:
%load_ext autoreload
%autoreload 2

sns.set_theme()
path = 'data' # relative path between parent directory (set above) to directory with saved data

mlflow.set_tracking_uri('http://127.0.0.1:5000')  # set up connection
mlflow.set_experiment('heracles_official')          # set the experiment

<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1681143470700, experiment_id='0', last_update_time=1681175451897, lifecycle_stage='active', name='heracles_official', tags={}>

# Util Functions

In [3]:
def generate_Q(num_sites, num_states):
    
    deletion_rate = 9e-4 # global deletion rate 
    mutation_rate = [0.1] * num_sites # site-specific mutation rate
    indel_distribution = [1/num_states] * num_states
    
    Q = torch.zeros(num_sites + num_states + 1, num_sites + num_states + 1)

    for i in range(num_sites + num_states): # fill in diagonals
        if i < num_sites:
            Q[i,i] = - (mutation_rate[i] + deletion_rate)
        else:
            Q[i,i] = - deletion_rate
            
    for i in range(num_sites): # fill in upper right
        for j in range(num_states):
            Q[i, num_sites + j] = mutation_rate[i] * indel_distribution[j]
            
    for i in range(num_sites + num_states):
        Q[i, -1] = deletion_rate
        
    return Q

# Running Model

In [4]:
# load true tree
fname = os.path.join(path, 'true_tree')
with open(fname, 'rb') as file:
    true_tree = pickle.load(file)
    
# load parameters for infinitesimal generator Q
fname = os.path.join(path, 'params')
with open(fname, 'rb') as file:
    params = pickle.load(file)

In [5]:
# hyper-params
char_matrix = true_tree.character_matrix.drop_duplicates().to_numpy(dtype=int) # character matrix of leaf cells
char_matrix = np.concatenate((char_matrix, np.zeros((1, char_matrix.shape[1]))), axis=0) # add unmodified cell

num_cells = char_matrix.shape[0] # number of cells
num_sites = char_matrix.shape[1] # number of target sites
num_states = 15
embedding_dim = 3

Q_list = [generate_Q(num_sites, num_states) for _ in range(num_sites)]
rho = torch.tensor([2], dtype=torch.float64)

In [12]:
dist_matrix = char_matrix_to_dist_matrix(char_matrix) # compute distance matrix
est_tree = estimate_tree(dist_matrix, method='neighbor-joining') # estimate phylogenetic tree
X = embed_tree(est_tree, torch.sqrt(rho), num_cells, local_dim=embedding_dim-1)

l = Logalike(X=X,
             priors=None,
             Q_list=Q_list,
             character_matrix=char_matrix,
             num_states=num_states,
             rho=rho,
            )

opt = geoopt.optim.RiemannianSGD([l.X], lr=5e-2, stabilize=1)
num_epochs = 30

loss_history, dist_error_history = [], []
for epoch in range(num_epochs):
    epoch_loss = 0

    for i in range(num_cells):
        opt.zero_grad() # zero gradient
        loss = -l.forward(i) # likelihood of tree configuration TODO: possibly negate
        loss.backward() # gradient on manifold
        opt.step() # take opt step
        
        epoch_loss += loss.item()
        
    log_metric('epoch_loss', epoch_loss)
    log_metric('pairwise dist correlation', dist_correlation(true_tree, l.X, rho))
    log_metric('robinson_foulds', rf(true_tree, l.X, rho))
    log_metric('triplets correct', tc2(true_tree, l.X, rho))
    
    loss_history.append(epoch_loss)

/Users/gil2rok/mambaforge/envs/crispr-phylogeny/lib/python3.10/site-packages/cassiopeia/data/utilities.py:166: CassiopeiaTreeWarning: Failed to numbaize dissimilarity function. Falling back to Python.
  warnings.warn(
/Users/gil2rok/school/crispr-phylogeny2/code/Heracles/src/metrics.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  u = torch.tensor(u)
/Users/gil2rok/school/crispr-phylogeny2/code/Heracles/src/metrics.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  v = torch.tensor(v)


est_dist (21, 21) 
 [[0.00000000e+00 4.47213592e-08 4.47213592e-08 2.72429333e+00
  2.72429333e+00 2.72429333e+00 1.69067140e+00 4.01795059e+00
  3.65475854e+00 4.89760125e+00 4.89760125e+00 4.89760125e+00
  4.89760125e+00 4.89760125e+00 4.89760125e+00 3.99140302e+00
  3.99140302e+00 3.99140302e+00 1.28822043e+00 2.12291438e+00
  1.00517926e+00]
 [4.47213592e-08 0.00000000e+00 4.47213592e-08 2.72429333e+00
  2.72429333e+00 2.72429333e+00 1.69067140e+00 4.01795059e+00
  3.65475854e+00 4.89760125e+00 4.89760125e+00 4.89760125e+00
  4.89760125e+00 4.89760125e+00 4.89760125e+00 3.99140302e+00
  3.99140302e+00 3.99140302e+00 1.28822043e+00 2.12291438e+00
  1.00517926e+00]
 [4.47213592e-08 4.47213592e-08 0.00000000e+00 2.72429333e+00
  2.72429333e+00 2.72429333e+00 1.69067140e+00 4.01795059e+00
  3.65475854e+00 4.89760125e+00 4.89760125e+00 4.89760125e+00
  4.89760125e+00 4.89760125e+00 4.89760125e+00 3.99140302e+00
  3.99140302e+00 3.99140302e+00 1.28822043e+00 2.12291438e+00
  1.00517926e+

/Users/gil2rok/mambaforge/envs/crispr-phylogeny/lib/python3.10/site-packages/cassiopeia/data/utilities.py:166: CassiopeiaTreeWarning: Failed to numbaize dissimilarity function. Falling back to Python.
  warnings.warn(
/Users/gil2rok/school/crispr-phylogeny2/code/Heracles/src/metrics.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  u = torch.tensor(u)
/Users/gil2rok/school/crispr-phylogeny2/code/Heracles/src/metrics.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  v = torch.tensor(v)


est_dist (21, 21) 
 [[0.00000000e+00 4.47213592e-08 4.47213592e-08 3.59945811e+00
  3.59945811e+00 3.59945811e+00 2.19084889e+00 4.60364225e+00
  4.27931988e+00 5.15097573e+00 5.15097573e+00 5.15097573e+00
  5.15097573e+00 5.15097573e+00 5.15097573e+00 5.36087171e+00
  5.36087171e+00 5.36087171e+00 1.55929533e+00 2.73411614e+00
  1.22744527e+00]
 [4.47213592e-08 0.00000000e+00 4.47213592e-08 3.59945811e+00
  3.59945811e+00 3.59945811e+00 2.19084889e+00 4.60364225e+00
  4.27931988e+00 5.15097573e+00 5.15097573e+00 5.15097573e+00
  5.15097573e+00 5.15097573e+00 5.15097573e+00 5.36087171e+00
  5.36087171e+00 5.36087171e+00 1.55929533e+00 2.73411614e+00
  1.22744527e+00]
 [4.47213592e-08 4.47213592e-08 0.00000000e+00 3.59945811e+00
  3.59945811e+00 3.59945811e+00 2.19084889e+00 4.60364225e+00
  4.27931988e+00 5.15097573e+00 5.15097573e+00 5.15097573e+00
  5.15097573e+00 5.15097573e+00 5.15097573e+00 5.36087171e+00
  5.36087171e+00 5.36087171e+00 1.55929533e+00 2.73411614e+00
  1.22744527e+

/Users/gil2rok/mambaforge/envs/crispr-phylogeny/lib/python3.10/site-packages/cassiopeia/data/utilities.py:166: CassiopeiaTreeWarning: Failed to numbaize dissimilarity function. Falling back to Python.
  warnings.warn(
/Users/gil2rok/school/crispr-phylogeny2/code/Heracles/src/metrics.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  u = torch.tensor(u)
/Users/gil2rok/school/crispr-phylogeny2/code/Heracles/src/metrics.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  v = torch.tensor(v)


...

/Users/gil2rok/mambaforge/envs/crispr-phylogeny/lib/python3.10/site-packages/cassiopeia/data/utilities.py:166: CassiopeiaTreeWarning: Failed to numbaize dissimilarity function. Falling back to Python.
  warnings.warn(
/Users/gil2rok/school/crispr-phylogeny2/code/Heracles/src/metrics.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  u = torch.tensor(u)
/Users/gil2rok/school/crispr-phylogeny2/code/Heracles/src/metrics.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  v = torch.tensor(v)


est_dist (21, 21) 
 [[0.00000000e+00 1.19209289e-07 1.19209289e-07 1.08834753e+01
  1.08834753e+01 1.08834753e+01 5.42556896e+00 7.77833585e+00
  7.66278281e+00 5.71628455e+00 5.71628455e+00 5.71628455e+00
  5.71628455e+00 5.71628455e+00 5.71628455e+00 1.69842726e+01
  1.69842726e+01 1.69842726e+01 3.09974093e+00 1.20617221e+01
  2.56343258e+00]
 [1.19209289e-07 0.00000000e+00 1.19209289e-07 1.08834753e+01
  1.08834753e+01 1.08834753e+01 5.42556896e+00 7.77833585e+00
  7.66278281e+00 5.71628455e+00 5.71628455e+00 5.71628455e+00
  5.71628455e+00 5.71628455e+00 5.71628455e+00 1.69842726e+01
  1.69842726e+01 1.69842726e+01 3.09974093e+00 1.20617221e+01
  2.56343258e+00]
 [1.19209289e-07 1.19209289e-07 0.00000000e+00 1.08834753e+01
  1.08834753e+01 1.08834753e+01 5.42556896e+00 7.77833585e+00
  7.66278281e+00 5.71628455e+00 5.71628455e+00 5.71628455e+00
  5.71628455e+00 5.71628455e+00 5.71628455e+00 1.69842726e+01
  1.69842726e+01 1.69842726e+01 3.09974093e+00 1.20617221e+01
  2.56343258e+

/Users/gil2rok/mambaforge/envs/crispr-phylogeny/lib/python3.10/site-packages/cassiopeia/data/utilities.py:166: CassiopeiaTreeWarning: Failed to numbaize dissimilarity function. Falling back to Python.
  warnings.warn(


est_dist (21, 21) 
 [[0.00000000e+00 4.47213589e-08 4.47213589e-08 1.10224443e+01
  1.10224443e+01 1.10224443e+01 5.47144356e+00 7.81497630e+00
  7.70513473e+00 5.68101509e+00 5.68101509e+00 5.68101509e+00
  5.68101509e+00 5.68101509e+00 5.68101509e+00 1.72286015e+01
  1.72286015e+01 1.72286015e+01 3.13727314e+00 1.22413975e+01
  2.59111401e+00]
 [4.47213589e-08 0.00000000e+00 4.47213589e-08 1.10224443e+01
  1.10224443e+01 1.10224443e+01 5.47144356e+00 7.81497630e+00
  7.70513473e+00 5.68101509e+00 5.68101509e+00 5.68101509e+00
  5.68101509e+00 5.68101509e+00 5.68101509e+00 1.72286015e+01
  1.72286015e+01 1.72286015e+01 3.13727314e+00 1.22413975e+01
  2.59111401e+00]
 [4.47213589e-08 4.47213589e-08 0.00000000e+00 1.10224443e+01
  1.10224443e+01 1.10224443e+01 5.47144356e+00 7.81497630e+00
  7.70513473e+00 5.68101509e+00 5.68101509e+00 5.68101509e+00
  5.68101509e+00 5.68101509e+00 5.68101509e+00 1.72286015e+01
  1.72286015e+01 1.72286015e+01 3.13727314e+00 1.22413975e+01
  2.59111401e+

/Users/gil2rok/school/crispr-phylogeny2/code/Heracles/src/metrics.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  u = torch.tensor(u)
/Users/gil2rok/school/crispr-phylogeny2/code/Heracles/src/metrics.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  v = torch.tensor(v)
